# SensoryForge Quickstart

This notebook demonstrates basic usage of SensoryForge for tactile sensory encoding.

**What you'll learn:**
- Creating a spatial grid
- Generating a simple stimulus
- Creating receptive fields (innervation)
- Applying SA/RA temporal filters
- Running spiking neuron models

## Setup

In [ ]:
import torch
import matplotlib.pyplot as plt

from sensoryforge.core import create_grid_torch
from sensoryforge.stimuli import gaussian_pressure_torch
from sensoryforge.core.innervation import create_sa_innervation
from sensoryforge.filters import CombinedSARAFilter
from sensoryforge.neurons import IzhikevichNeuronTorch

# Set device
device = 'cpu'  # Change to 'cuda' if GPU available
print(f"Using device: {device}")

## Step 1: Create Spatial Grid

The grid represents the sensor array (e.g., tactile sensor surface).

In [ ]:
# Create 40x40 grid with 0.15mm spacing
xx, yy, x_vec, y_vec = create_grid_torch(
    grid_size=40,
    spacing=0.15,
    center=(0.0, 0.0),
    device=device
)

print(f"Grid shape: {xx.shape}")
print(f"Spatial extent: x=[{x_vec.min():.2f}, {x_vec.max():.2f}] mm, y=[{y_vec.min():.2f}, {y_vec.max():.2f}] mm")

## Step 2: Generate Stimulus

Create a Gaussian pressure pattern.

In [ ]:
# Generate Gaussian stimulus at center
stimulus = gaussian_pressure_torch(
    xx, yy,
    center_x=0.0,
    center_y=0.0,
    amplitude=1.0,
    sigma=0.3  # 0.3 mm spread
)

# Visualize
plt.figure(figsize=(6, 5))
plt.imshow(stimulus.cpu().numpy(), extent=[x_vec.min(), x_vec.max(), y_vec.min(), y_vec.max()], 
           origin='lower', cmap='viridis')
plt.colorbar(label='Pressure (normalized)')
plt.xlabel('x (mm)')
plt.ylabel('y (mm)')
plt.title('Gaussian Stimulus')
plt.show()

print(f"Stimulus shape: {stimulus.shape}")
print(f"Peak pressure: {stimulus.max().item():.3f}")

## Step 3: Create Innervation (Receptive Fields)

Each neuron receives weighted input from multiple grid points.

In [ ]:
# Create innervation for 50 SA neurons
num_neurons = 50
xlim = (x_vec.min().item(), x_vec.max().item())
ylim = (y_vec.min().item(), y_vec.max().item())

innervation_weights, neuron_centers = create_sa_innervation(
    num_neurons=num_neurons,
    xx=xx,
    yy=yy,
    xlim=xlim,
    ylim=ylim,
    sigma_mm=0.5,  # Receptive field spread
    device=device
)

print(f"Innervation weights shape: {innervation_weights.shape}")
print(f"Neuron centers shape: {neuron_centers.shape}")

# Compute neural responses (weighted sum over receptive fields)
# innervation_weights: [num_neurons, grid_h, grid_w]
# stimulus: [grid_h, grid_w]
neural_input = (innervation_weights * stimulus.unsqueeze(0)).sum(dim=(1, 2))

print(f"Neural input shape: {neural_input.shape}")
print(f"Mean activation: {neural_input.mean().item():.4f}")

## Step 4: Apply Temporal Filters

SA/RA filters decompose the signal into sustained and transient components.

In [ ]:
# Create combined SA/RA filter
filter = CombinedSARAFilter(
    num_neurons=num_neurons,
    device=device
)

# Add batch dimension: [1, num_neurons]
neural_input_batch = neural_input.unsqueeze(0)

# Apply filter (single step, steady-state mode)
sa_current, ra_current = filter(neural_input_batch, mode='steady_state')

print(f"SA current shape: {sa_current.shape}")
print(f"RA current shape: {ra_current.shape}")
print(f"SA mean: {sa_current.mean().item():.4f} mA")
print(f"RA mean: {ra_current.mean().item():.4f} mA")

## Step 5: Generate Spikes

Convert filtered currents to spike trains using Izhikevich neurons.

In [ ]:
# Create SA neurons
sa_neurons = IzhikevichNeuronTorch(
    num_neurons=num_neurons,
    device=device
)

# Simulate for 100 time steps
dt_ms = 0.1
num_steps = 100
spikes_list = []

# Reset neurons
sa_neurons.reset()

# Run simulation
for t in range(num_steps):
    spikes, state = sa_neurons(sa_current)
    spikes_list.append(spikes)

# Stack spikes: [num_steps, batch, num_neurons]
spike_train = torch.stack(spikes_list, dim=0)

print(f"Spike train shape: {spike_train.shape}")
print(f"Total spikes: {spike_train.sum().item():.0f}")
print(f"Mean firing rate: {spike_train.sum().item() / (num_neurons * num_steps * dt_ms) * 1000:.2f} Hz")

## Visualize Spike Raster

In [ ]:
# Convert to numpy for plotting
spike_train_np = spike_train.squeeze(1).cpu().numpy()  # [num_steps, num_neurons]

# Find spike times and neuron indices
spike_times, neuron_ids = torch.where(torch.from_numpy(spike_train_np) > 0.5)
spike_times = spike_times.numpy() * dt_ms  # Convert to ms
neuron_ids = neuron_ids.numpy()

# Plot raster
plt.figure(figsize=(10, 6))
plt.scatter(spike_times, neuron_ids, s=1, color='black')
plt.xlabel('Time (ms)')
plt.ylabel('Neuron ID')
plt.title(f'Spike Raster Plot ({len(spike_times)} spikes)')
plt.xlim(0, num_steps * dt_ms)
plt.ylim(-1, num_neurons)
plt.show()

## Summary

You've completed a basic SensoryForge encoding pipeline:

1. ✅ Created a spatial grid
2. ✅ Generated a Gaussian stimulus
3. ✅ Created receptive field innervation
4. ✅ Applied SA/RA temporal filters
5. ✅ Generated spike trains with spiking neurons

**Next steps:**
- Try different stimulus patterns (moving, textures)
- Experiment with different neuron models (AdEx, MQIF)
- Create multi-population grids
- Build complete pipelines with configuration files